In [1]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import lightning.pytorch as pl
from lightning.pytorch.callbacks import RichProgressBar, Timer

# Add the prime_torch file to the system path so we can import it
import sys
sys.path.append("/glade/u/home/cobrien/prime/prime_lib/primesw")
from data import SWDataset, SWDataModule
from prime_torch import crps, SWRegressor

DATAPATH = '~/data/'

In [2]:
# test_dataframe = pd.DataFrame([])
# test_dataframe['time'] = pd.date_range(
#     pd.to_datetime('20150902 00:00:00+0000'),
#     pd.to_datetime('20250101 00:00:00+0000'),
#     freq = '100s'
# )
# test_dataframe['a'] = np.arange(len(test_dataframe)) # Fake input
# test_dataframe['b'] = test_dataframe['a'] * 2 # Fake target
# test_dataframe['x_pos'] = np.arange(len(test_dataframe)) * 0.1
# test_dataframe['y_pos'] = np.arange(len(test_dataframe)) * 0.2
# test_dataframe['z_pos'] = np.arange(len(test_dataframe)) * 0.3
# test_dataframe.to_hdf("~/data/prime/test.h5", key = 'lineartest')

data = pd.read_hdf(DATAPATH + 'combined_data.h5', key = '1min_mms_wind')
# list(data.columns)

In [3]:
data.loc[(data['modified_named_label'] == 'solar wind')&(data['stable'] == 1), :].iloc[int(0.8*247215):, :]

,Epoch,probe,ratio_max_width,ratio_high_low,norm_Btot,small_energy_mean,large_energy_mean,temp_total,r_gse_x,r_gse_y,...,BGSE_0,BGSE_1,BGSE_2,PGSM_0,PGSM_1,PGSM_2,PGSE_0,PGSE_1,PGSE_2,stable
783158,2022-12-06 11:22:00+00:00,mms1,0.099113,0.972141,0.090599,4.149168,4.033574,43.810337,70372.151606,121291.038427,...,-4.115065,1.472905,-0.492083,220.266197,98.199184,-26.094267,220.266197,101.554920,3.253949,1.0
783159,2022-12-06 11:23:00+00:00,mms1,0.099515,0.948516,0.091167,4.146450,3.932973,43.306786,70376.310770,121358.080725,...,-4.138211,1.462516,-0.525947,220.267014,98.203819,-26.075378,220.267014,101.554546,3.254244,1.0
783160,2022-12-06 11:24:00+00:00,mms1,0.096700,0.951711,0.090902,4.122525,3.923451,41.421787,70380.430313,121425.050925,...,-4.107340,1.537073,-0.616459,220.267830,98.208530,-26.056184,220.267830,101.554169,3.254539,1.0
783161,2022-12-06 11:25:00+00:00,mms1,0.097986,0.933466,0.090189,4.155700,3.879203,41.902737,70384.521589,121491.969787,...,-4.101038,1.556031,-0.626554,220.268639,98.213322,-26.036689,220.268639,101.553791,3.254835,1.0
783162,2022-12-06 11:26:00+00:00,mms1,0.097573,0.931189,0.089900,4.113208,3.830173,41.628290,70388.573237,121558.816731,...,-4.058907,1.623733,-0.725471,220.269447,98.218189,-26.016889,220.269447,101.553413,3.255130,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905237,2023-06-01 14:17:00+00:00,mms1,0.139900,1.053616,0.119622,3.972942,4.185954,84.981760,6950.367742,-140112.240351,...,3.125178,-1.409303,2.854743,227.914986,103.248070,5.631277,227.914986,103.337818,3.629431,1.0
905238,2023-06-01 14:18:00+00:00,mms1,0.137356,1.000378,0.118647,4.006738,4.008254,68.139725,6977.824836,-140057.052744,...,3.847021,-0.835190,2.378296,227.915802,103.244072,5.696580,227.915802,103.337395,3.629699,1.0
905239,2023-06-01 14:19:00+00:00,mms1,0.140705,0.994115,0.120214,4.052110,4.028265,69.984500,7005.280935,-140001.818190,...,4.275983,-0.737198,1.802373,227.916618,103.240017,5.762163,227.916618,103.336967,3.629966,1.0
905240,2023-06-01 14:20:00+00:00,mms1,0.144459,1.117512,0.129521,3.667746,4.098751,95.530670,7032.735244,-139946.518416,...,4.273108,-1.447723,1.191945,227.917427,103.235905,5.828058,227.917427,103.336544,3.630233,1.0


In [4]:
target_features = [
    'mms1_dis_bulkv_gse_fast_0', # V GSE X
    'mms1_dis_bulkv_gse_fast_1', # V GSE Y
    'mms1_dis_bulkv_gse_fast_2', # V GSE Z
    # 'mms1_dis_numberdensity_fast', # Ni
    # 'mms1_dis_temppara_fast', # Ti parallel to B
    # 'mms1_dis_tempperp_fast', # Ti perpendicular to B
    'mms1_des_numberdensity_fast', # Ne
    'mms1_fgm_b_gse_srvy_l2_0', # B GSE X
    'mms1_fgm_b_gse_srvy_l2_1', # B GSE Y
    'mms1_fgm_b_gse_srvy_l2_2', # B GSE Z
]
input_features = [
    'Np', # Ni
    'V_GSE_0', # V GSE X
    'V_GSE_1', # V GSE Y
    'V_GSE_2', # V GSE Z
    'THERMAL_SPD', # Vth
    'BGSE_0', # B GSE X
    'BGSE_1', # B GSE Y
    'BGSE_2', # B GSE Z
    'PGSE_0', # Wind Position GSE X
    'PGSE_1', # Wind Position GSE Y
    'PGSE_2', # Wind Position GSE Z
]
position_features = [
    'mms1_mec_r_gse_0', # MMS Position GSE X
    'mms1_mec_r_gse_1', # MMS Position GSE Y
    'mms1_mec_r_gse_2', # MMS Position GSE Z
]

trn_bounds = ['20151007 12:08:00+0000', '20211217 04:33:00+0000']
tst_bounds = ['20211217 04:34:00+0000', '20221206 11:21:00+0000']
val_bounds = ['20221206 11:22:00+0000', '20230601 14:21:00+0000']
datamodule = SWDataModule(
    target_features = target_features,
    input_features = input_features,
    position_features = position_features,
    region = 'solar wind',
    cadence = '100s',
    window = 100,
    stride = 10,
    interp_frac = 0.1,
    trn_bounds = trn_bounds,
    val_bounds = val_bounds,
    tst_bounds = tst_bounds,
    datastore = "~/data/combined_data.h5",
    key = "1min_mms_wind",
)
datamodule.setup()

TypeError: SWDataModule.__init__() got an unexpected keyword argument 'region'

In [ ]:
model = SWRegressor(
    in_dim = len(input_features),
    tar_dim = len(target_features),
    pos_dim = len(position_features),
    decoder_type = 'linear',
    encoder_type = 'rnn',
    lr_scheduler = 'cosine',
    decoder_hidden_layers = [4],
    encoder_hidden_dim = 4,
    pos_encoding_size=4,
    encoder_num_layers=1,
    loss='mae'
)

/glade/work/cobrien/conda-envs/pt212gpu_conda/lib/python3.11/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
logger = pl.loggers.TensorBoardLogger(
    save_dir = "~/data/prime/tensorboard_logs",
    name = 'test_model',
)

In [ ]:
trainer = pl.Trainer(
    accelerator='cpu',
    max_epochs=1,
    callbacks = [
        Timer(), 
        RichProgressBar(),
        # ModelCheckpoint(),
        ],
    logger = logger,
    # precision='16-true', #Lower the precision to not blow up memory
)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/glade/work/cobrien/conda-envs/pt212gpu_conda/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/glade/work/cobrien/conda-envs/pt212gpu_conda/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip inst

In [ ]:
trainer.fit(model=model, datamodule=datamodule)

2025-09-25 08:26:33.253 | INFO     | data:setup:241 - Train dataloader is ready. Dataset size: 148220
2025-09-25 08:26:40.793 | INFO     | data:setup:259 - Validation dataloader is ready. Dataset size: 49334
2025-09-25 08:26:48.586 | INFO     | data:setup:277 - Test dataloader is ready. Dataset size: 49334


┏━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name    ┃ Type          ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ decoder │ LinearDecoder │    175 │ train │
└───┴─────────┴───────────────┴────────┴───────┘

Trainable params: 175                                                                                              
Non-trainable params: 0                                                                                            
Total params: 175                                                                                                  
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 6                                                                                           
Modules in eval mode: 0

Output()

/glade/work/cobrien/conda-envs/pt212gpu_conda/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/dat
a_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing 
the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.

/glade/work/cobrien/conda-envs/pt212gpu_conda/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/dat
a_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider 
increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve 
performance.

AttributeError: 'RecurrentEncoder' object has no attribute 'named_parameters'

In [ ]:
in_test = torch.rand((50,100,14))
tar_test = torch.rand((50,1))
pos_test = torch.rand((50,3))
out_test = model.forward(in_test, pos_test)
model.loss_fn(out_test, tar_test)

torch.Size([50, 100, 128])